### 1. Kütüphanelerin Import Edilmesi

In [ ]:
import pandas as pd
import numpy as np  
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder , RobustScaler

print("kutuphaneler yuklendi")

### 2. Kesifsel Veri Analizi (EDA)

In [ ]:
df = pd.read_csv("../data/raw/german_credit_data.csv", index_col=0)

print("(satir, sutun) sayisi:", df.shape)
print("ilk 5 satir:")
df.head() 

In [ ]:
print("Veri Tipleri ve Genel Bilgi:")
df.info()
print("\nİstatistiksel Özet:")
df.describe(include='all')

In [ ]:
# Hedef değişkenin dağılımına bakma
fig = px.histogram(df,
                    x='Risk',
                    title='Hedef Değişken Dağılımı',
                    labels={'Risk': 'Kredi Riski'},
                    color='Risk',
                    color_discrete_map={
                        'Good': 'lightblue',
                        'Bad': '#FF0000'  
                    }
                   )
fig.show()


In [ ]:
# Risk'e göre sayısal değişkenlerin dağılımı 
numeric_cols = ['Age', 'Credit amount', 'Duration']

for col in numeric_cols:
    fig = px.box(df, x='Risk', y=col, 
                 color='Risk',
                 title=f'{col} - Risk Kategorilerine Göre Dağılım',
                 color_discrete_map={'good': '#2ecc71', 'bad': '#e74c3c'},
                 height=400)
    fig.show()

In [ ]:
# Risk'e göre kategorik değişkenlerin analizi 
categorical_cols = ['Sex', 'Job', 'Housing', 'Purpose']

for cat_col in categorical_cols:
    # Crosstab oluştur
    cross_tab = pd.crosstab(df[cat_col], df['Risk'])
    cross_tab_reset = cross_tab.reset_index()
    
    # Long format'a çevir
    cross_tab_long = cross_tab_reset.melt(id_vars=cat_col, 
                                           var_name='Risk', 
                                           value_name='Sayı')
    
    fig = px.bar(cross_tab_long, 
                 x=cat_col, 
                 y='Sayı', 
                 color='Risk',
                 title=f'{cat_col} - Risk Kategorilerine Göre Dağılım',
                 barmode='stack',
                 color_discrete_map={'good': '#2ecc71', 'bad': '#e74c3c'},
                 height=400)
    fig.show()

#### 2.1 Eksik Veri Analizi 

In [ ]:
print("Eksik Değerler:\n", df.isnull().sum())
print("\nEksik Değer Yüzdeleri:\n", (df.isnull().mean() * 100))


In [ ]:
# Eksik veri olan kolonların görselleştirmesi 
na_cols = ['Saving accounts', 'Checking account']

for col in na_cols:
    # Geçici olarak NaN'ları göster
    temp_data = df[col].fillna('Missing/NA').value_counts().reset_index()
    temp_data.columns = ['Kategori', 'Sayı']
    
    fig = px.bar(temp_data, 
                 x='Kategori', 
                 y='Sayı',
                 title=f'{col} - Veri Dağılımı',
                 color='Sayı',
                 color_continuous_scale='Viridis')
    fig.show()

#### 2.2 Aykiri Değer Analizi

In [ ]:
# IQR yöntemiyle aykırı değer tespiti
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("IQR Yöntemiyle Aykırı Değer Analizi:")
print("="*60)
for col in numeric_cols:
    n_outliers, lower, upper = detect_outliers_iqr(df, col)
    print(f"{col}:")
    print(f"  Alt Sınır: {lower:.2f}, Üst Sınır: {upper:.2f}")
    print(f"  Aykırı Değer Sayısı: {n_outliers} ({n_outliers/len(df)*100:.2f}%)")
    print()

**Yorum** Finansal verilerde aykırı değerler gerçek durumları yansıtabilir (örneğin, çok yüksek kredi tutarları). Bu yüzden aykırı değerleri silmek yerine, ölçekleme sırasında `RobustScaler` kullanacağım.

In [ ]:
# Aykiri degerlerin box plot ile görselleştirilmesi
df_melted = df[numeric_cols].melt(var_name='Değişken', value_name='Değer')

fig = px.box(df_melted, 
             y='Değer',
             facet_col='Değişken',
             title='Aykırı Değerlerin Box Plot ile Görselleştirilmesi',
             height=400)
fig.update_yaxes(matches=None)  # Her grafik kendi skalasını kullansın
fig.show()

### 3. VERİ ÖN İŞLEME (PREPROCESSING)

In [ ]:
# Orijinal veriyi korumak için kopyasını oluşturuyoruz
df_processed = df.copy()
print(f"Preprocessing için kopyalanan veri boyutu: {df_processed.shape}")

#### 3.1 Eksik Veri Isleme

In [ ]:
# 'NA' string değerlerini ve NaN değerlerini 'No Account' kategorisine çeviriyoruz
df_processed['Saving accounts'] = df_processed['Saving accounts'].replace('NA', 'No Account').fillna('No Account')
df_processed['Checking account'] = df_processed['Checking account'].replace('NA', 'No Account').fillna('No Account')

print("Eksik Veri İşleme Sonrası:")
print("\nSaving accounts:")
print(df_processed['Saving accounts'].value_counts())
print("\nChecking account:")
print(df_processed['Checking account'].value_counts())

#### 3.2 Hedef Değişken (Risk) Encoding

In [ ]:
# Risk değişkeni (good=0, bad=1)
df_processed['Risk'] = df_processed['Risk'].map({'good': 0, 'bad': 1})

print("Risk Encoding:")
print(df_processed['Risk'].value_counts())


#### 3.3 Train-Test Split (Data Leakage Onlendi)


In [ ]:
# X ve y ayrımı
X = df_processed.drop('Risk', axis=1)
y = df_processed['Risk']

# Train-Test split (%80 train, %20 test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train Set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df_processed)*100:.1f}%)")
print(f"Test Set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df_processed)*100:.1f}%)")
print(f"\nTrain Set Risk Dağılımı:")
print(y_train.value_counts())
print(f"\nTest Set Risk Dağılımı:")
print(y_test.value_counts())

#### 3.4 Ordinal Encoding 

In [ ]:
# Saving accounts ve Checking account sıralı (ordinal) değişkenler
# No Account < little < moderate < quite rich/rich

ordinal_mapping = {
    'Saving accounts': ['No Account', 'little', 'moderate', 'quite rich', 'rich'],
    'Checking account': ['No Account', 'little', 'moderate', 'rich']
}

# Ordinal Encoder oluşturuldu
ordinal_encoder = OrdinalEncoder(categories=[ordinal_mapping['Saving accounts'], 
                                              ordinal_mapping['Checking account']])

X_train[['Saving accounts', 'Checking account']] = ordinal_encoder.fit_transform(
    X_train[['Saving accounts', 'Checking account']]
)

X_test[['Saving accounts', 'Checking account']] = ordinal_encoder.transform(
    X_test[['Saving accounts', 'Checking account']]
)
# ordinal encoding train ve test setlerine uygulandı

print("Ordinal Encoding tamamlandı!")
print("\nSaving accounts encoding:")
for i, cat in enumerate(ordinal_mapping['Saving accounts']):
    print(f"  {cat} -> {i}")
print("\nChecking account encoding:")
for i, cat in enumerate(ordinal_mapping['Checking account']):
    print(f"  {cat} -> {i}")

#### 3.5 One-Hot Encoding 

In [ ]:
# Kategorik değişkenler için One-Hot Encoding
# drop_first=True ile Dummy Variable Trap'ten kaçınıyorum

categorical_cols_for_ohe = ['Sex', 'Job', 'Housing', 'Purpose']

X_train = pd.get_dummies(X_train, columns=categorical_cols_for_ohe, drop_first=True)

X_test = pd.get_dummies(X_test, columns=categorical_cols_for_ohe, drop_first=True)

# Sütunların aynı olduğundan emin olmak icin
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print(f"One-Hot Encoding tamamlandı!")
print(f"Train set şekli: {X_train.shape}")
print(f"Test set şekli: {X_test.shape}")
print(f"\nYeni öznitelik sayısı: {X_train.shape[1]}")

#### 3.6 Olceklendirme (Scailing)

In [ ]:
# Aykırı değerler olduğu için RobustScaler kullanıyorum
scaler = RobustScaler()

# Ölçeklendirilecek sayısal sütunlar
numeric_cols_to_scale = ['Age', 'Credit amount', 'Duration']

# Train seti üzerinde fit ve transform
X_train[numeric_cols_to_scale] = scaler.fit_transform(X_train[numeric_cols_to_scale])

# Test seti üzerinde sadece transform
X_test[numeric_cols_to_scale] = scaler.transform(X_test[numeric_cols_to_scale])

print("RobustScaler ile ölçeklendirme tamamlandı!")
print(f"\nÖlçeklendirilen sütunlar: {numeric_cols_to_scale}")
print(f"\nTrain set istatistikleri:")
print(X_train[numeric_cols_to_scale].describe())

#### 3.7 İşlenmiş Verinin Kaydedilmesi

In [ ]:
# Train ve test setlerini kaydet
import os
os.makedirs('../data/processed', exist_ok=True)

X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("İşlenmiş veri setleri kaydedildi!")
print(f"\nKaydedilen dosyalar:")
print("  - ../data/processed/X_train.csv")
print("  - ../data/processed/X_test.csv")
print("  - ../data/processed/y_train.csv")
print("  - ../data/processed/y_test.csv")

### 4. Ozet ve Sonuclar

In [ ]:
print("="*70)
print("EDA ve ÖN İŞLEME ÖZET RAPORU")
print("="*70)

print("\n1. VERİ SETİ BİLGİLERİ:")
print(f"   - Toplam Örnek Sayısı: {len(df)}")
print(f"   - Öznitelik Sayısı (Orijinal): {df.shape[1]}")
print(f"   - Öznitelik Sayısı (İşlenmiş): {X_train.shape[1]}")

print("\n2. HEDEF DEĞİŞKEN (RISK) DAĞILIMI:")
print(f"   - Good Risk: {(y==0).sum()} (%{(y==0).sum()/len(y)*100:.1f})")
print(f"   - Bad Risk: {(y==1).sum()} (%{(y==1).sum()/len(y)*100:.1f})")

print("\n3. EKSİK VERİ İŞLEME:")
print(f"   - Saving accounts: 'NA' değerleri 'No Account' olarak işlendi")
print(f"   - Checking account: 'NA' değerleri 'No Account' olarak işlendi")

print("\n4. AYKIRI DEĞER ANALİZİ:")
print(f"   - Credit amount, Age, Duration değişkenlerinde aykırı değer tespit edildi")
print(f"   - Aykırı değerler korundu, RobustScaler ile ölçekleme yapıldı")

print("\n5. ENCODING İŞLEMLERİ:")
print(f"   - Risk: Label Encoding (good=0, bad=1)")
print(f"   - Saving/Checking accounts: Ordinal Encoding")
print(f"   - Sex, Job, Housing, Purpose: One-Hot Encoding (drop_first=True)")

print("\n6. ÖLÇEKLEME:")
print(f"   - Yöntem: RobustScaler")
print(f"   - Ölçeklenen değişkenler: Age, Credit amount, Duration")

print("\n7. TRAIN-TEST SPLIT:")
print(f"   - Train Set: {X_train.shape[0]} samples (%80)")
print(f"   - Test Set: {X_test.shape[0]} samples (%20)")
print(f"   - Stratify: Evet (Risk değişkenine göre)")

print("\n8. FİNAL VERİ SETİ:")
print(f"   - X_train: {X_train.shape}")
print(f"   - X_test: {X_test.shape}")
print(f"   - y_train: {y_train.shape}")
print(f"   - y_test: {y_test.shape}")

print("\n" + "="*70)
print("Veri hazır! Bir sonraki adım: Öznitelik Seçimi (Feature Selection)")
print("="*70)